<a href="https://colab.research.google.com/github/Banafshehkh/Fake_News_Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Author: Banafsheh Khazali
#Date: April 26, 2023

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **Libraries**

In [7]:
import pandas as pd
import numpy as np
import os
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
train_data = pd.read_csv("/content/drive/MyDrive/train.csv")
train_data.head(5)

,Statement,Label
0,Says the Annies List political group supports ...,False
1,When did the decline of coal start? It started...,True
2,"Hillary Clinton agrees with John McCain ""by vo...",True
3,Health care reform legislation is likely to ma...,False
4,The economic turnaround started at the end of ...,True


## **Preprocessing**

In this step, preprocessing steps are done:


*   lowercasing
*   tokenizing and stemming
*   removing punctuations
*   removing stop word
*   removing null values
*   removing html tags



In [41]:
def preprocessing_pipeline(data_folder):
  csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
  stop_words = set(stopwords.words('english'))
  ps = PorterStemmer()

  punc = set(string.punctuation)

  for csv_file in csv_files:
    csv_path = os.path.join(data_folder,csv_file)
    df = pd.read_csv(csv_path)

    #make everything lowercase
    df.iloc[:, 0] = df.iloc[:, 0].str.lower() 
    # remove stop words
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
    # remove punctuations
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: " ".join([char for char in x.split() if char not in punc ]))
    #remove html tags
    df.iloc[:, 0] = df.iloc[:,0].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
    #tokenizing and stemming
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: word_tokenize(x.lower()))
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: [ps.stem(token) for token in x])
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: " ".join(x))
    # remove missing values
    df = df.dropna()

    # display(df)

    #save preprosessed files into a new one
    new_csv_file = os.path.splitext(csv_file)[0] + 'preprocessed.csv'
    new_csv_path = os.path.join(data_folder, new_csv_file)
    df.to_csv(new_csv_path, index=False)


preprocessing_pipeline("/content/drive/MyDrive/Data")

<ipython-input-41-9992479baa52>:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df.iloc[:, 0] = df.iloc[:,0].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
<ipython-input-41-9992479baa52>:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df.iloc[:, 0] = df.iloc[:,0].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
<ipython-input-41-9992479baa52>:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df.iloc[:, 0] = df.iloc[:,0].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
<ipython-input-41-9992479baa52>:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file 